In [7]:
import numpy as np
import torch
from torch import nn
import torch.nn.functional as F
import gymnasium as gym
from gymnasium.wrappers import RecordVideo
from collections import Counter
from itertools import count

In [3]:
class DeepQNetwork(nn.Module):
    def __init__(self, n_observations, n_actions):
        super(DeepQNetwork, self).__init__()
        self.layer1 = nn.Linear(n_observations, 128)
        self.layer2 = nn.Linear(128, 128)
        self.layer3 = nn.Linear(128, n_actions)

    def forward(self, x):
        x = F.relu(self.layer1(x))
        x = F.relu(self.layer2(x))
        return self.layer3(x)

In [1]:
dqn = DeepQNetwork(4, 2)
# dqn.load_state_dict(torch.load("../models/cartpole-gorila.pth"))
# dqn.load_state_dict(torch.load("../checkpoints/checkpoint-250000.pth"))
# dqn.load_state_dict(torch.load("../checkpoints/checkpoint-87500.pth"))
dqn.load_state_dict(torch.load("../models/checkpoint-250000.pth"))

NameError: name 'DeepQNetwork' is not defined

In [8]:
dqn = DeepQNetwork(4, 2)
dqn.load_state_dict(torch.load("../models/checkpoint-250000.pth"))

dqn.eval()
env = gym.make('CartPole-v1', render_mode="rgb_array")
env = RecordVideo(env, "videos")
state, _ = env.reset()
actions = []
rewards = 0

for _ in count():
    q_values = dqn(torch.tensor(np.array(state)).unsqueeze(0))
    action = q_values.argmax().item()
    actions.append(action)
    # print(action, q_values)
    state, reward, terminated, truncated, _ = env.step(action)
    rewards += reward
    if terminated or truncated:
        break

print("Total reward:", rewards)
print(Counter(actions))
env.close()

/home/jlcg/anaconda3/envs/dist/lib/python3.11/site-packages/gymnasium/wrappers/record_video.py:94: UserWarning: WARN: Overwriting existing videos at /home/jlcg/projects/distributed-reinforcement-learning/notebooks/videos folder (try specifying a different `video_folder` for the `RecordVideo` wrapper if this is not desired)
  logger.warn(


Moviepy - Building video /home/jlcg/projects/distributed-reinforcement-learning/notebooks/videos/rl-video-episode-0.mp4.
Moviepy - Writing video /home/jlcg/projects/distributed-reinforcement-learning/notebooks/videos/rl-video-episode-0.mp4



Moviepy - Done !
Moviepy - video ready /home/jlcg/projects/distributed-reinforcement-learning/notebooks/videos/rl-video-episode-0.mp4
Total reward: 481.0
Counter({1: 246, 0: 235})
